# Dask Array

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

In [2]:
import dask.array as da
import h5py
import numpy as np
import dask
import pandas as pd

In [2]:
dask.__version__

'2022.12.0'

In [2]:
!pip install graphviz 

     |████████████████████████████████| 47 kB 243 kB/s eta 0:00:01


## Лабораторная работа 11

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy`, `pandas` и `dask`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy`, структур `pandas` или структур `dask` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

В ходе выполнения все операции вычислений (расчет средних значений, расчет косинусной близости и т.д.) проводятся над `dask.array` и средствами пакета `dask`, если в задании не сказано обратного. Переход от `dask.array` к `numpy.array` или `pd.DataFrame` возможен исключительно для демонстрации результата в конце решения задачи. Если в задаче используются результаты выполнения предыдущих задач, то подразумевается, что вы используете результаты в виде `dask.array` (то есть то, что было получено до вызова `compute`, а не после).

1\. Считайте датасет `embeddings` из файла `recipe_embeddings.h5` в виде `dask.array`. Выведите на экран основную информацию о массиве: размер, форму, тип, количество и размер сегментов. 

In [3]:
hdf = h5py.File("recipe_embeddings.h5", "r")
dset = hdf['embeddings']
arr = da.from_array(dset)

In [4]:
arr

dask.array<array, shape=(1200000, 312), dtype=float32, chunksize=(107546, 312), chunktype=numpy.ndarray>

In [ ]:
#размер массива: 1.39 GiB
#форма: (1200000, 312)
#тип: float32
#количество сегментов: 12
#размер сегментов: (100000, 312)

2\. Посчитайте и выведите на экран среднее значение всех элементов массива. Исследуйте, как влияет значение аргумента `chunks` при создании `dask.array` на скорость выполнения операции поиска среднего. 

Пусть $M$ - количество строк в массиве, $N$ - количество столбцов в массиве, `chunks=(r,c)`. Сравните несколько вариантов:
* $r=M$, $с \ll N$ , 
* $r \ll M$, $c=N$ 
* $r = M$, $c = N$ 
* значения $r, c$ по умолчанию.

Выберите наиболее оптимальные значения $r$ и  $c$ в смысле скорости вычислений и далее продолжайте работу с ними.

In [5]:
M = 1200000
N = 312

In [6]:
r = M
c = N // 12
arr_c_less = da.from_array(dset, chunks = (r,c))
arr_c_less

dask.array<array, shape=(1200000, 312), dtype=float32, chunksize=(1200000, 26), chunktype=numpy.ndarray>

In [6]:
r = M // 12
c = N
arr_r_less = da.from_array(dset, chunks = (r,c))
arr_r_less

dask.array<array, shape=(1200000, 312), dtype=float32, chunksize=(100000, 312), chunktype=numpy.ndarray>

In [8]:
r = M
c = N
arr_c_r_equal = da.from_array(dset, chunks = (r,c))
arr_c_r_equal

dask.array<array, shape=(1200000, 312), dtype=float32, chunksize=(1200000, 312), chunktype=numpy.ndarray>

In [11]:
%%time

m = arr.mean()
s = arr.sum()

m.compute(), s.compute()

CPU times: user 610 ms, sys: 1.15 s, total: 1.76 s
Wall time: 1.88 s


(0.0023777566, 890232.1)

In [12]:
%%time

m = arr_c_less.mean()
s = arr_c_less.sum()

m.compute(), s.compute()

CPU times: user 1.26 s, sys: 5.16 s, total: 6.42 s
Wall time: 6.2 s


(0.0023777548, 890231.4)

In [9]:
%%time

m = arr_r_less.mean()
s = arr_r_less.sum()

m.compute(), s.compute()

CPU times: user 586 ms, sys: 1.16 s, total: 1.75 s
Wall time: 2.02 s


(0.0023777566, 890232.1)

In [14]:
%%time

m = arr_c_r_equal.mean()
s = arr_c_r_equal.sum()

m.compute(), s.compute()

CPU times: user 510 ms, sys: 3.09 s, total: 3.6 s
Wall time: 5 s


(0.0023777678, 890236.25)

In [ ]:
#самый лучший результат у arr_r_less - будем им пользоваться

3\. Опишите пространство, в котором расположены эмбеддинги, посчитав минимальное и максимальное значение для каждой из координат. Сведите результаты в таблицу `pd.DataFrame`, состоящую из двух строк и 312 столбцов. Задайте индексы строк "min" и "max". Названия столбцов сделайте вида $x_i$. Выведите полученную таблицу на экран.

Решите задачу двумя способами. В первом варианте сделайте два вызова метода `compute` для расчета каждого из векторов максимальных и минимальных значений. Во втором варианте сделайте один вызов функции `dask.compute` для одновременного расчета двух векторов. Сравните время выполнения двух решений.

In [10]:
min_for_arr = arr_r_less.min(axis = 0)
min_for_arr.compute()[:5]

array([-0.13280304, -0.14905587, -0.09446791, -0.1916968 , -0.11422908],
      dtype=float32)

In [11]:
max_for_arr = arr_r_less.max(axis = 0)
max_for_arr.compute()[:5]

array([0.13503821, 0.07612539, 0.1578539 , 0.03098669, 0.10119153],
      dtype=float32)

In [12]:
columns_for_df = ['x{}'.format(i) for i in range(len(max_for_arr.compute()))]

In [13]:
pd.DataFrame(data = [min_for_arr.compute(), max_for_arr.compute()],
             columns = columns_for_df,
             index = ['min', 'max'])

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x302,x303,x304,x305,x306,x307,x308,x309,x310,x311
min,-0.132803,-0.149056,-0.094468,-0.191697,-0.114229,-0.114341,-0.096039,-0.115178,-0.157275,-0.116715,...,-0.103254,-0.122285,-0.149789,-0.127703,-0.094802,-0.119690,-0.141425,-0.123732,-0.081543,-0.227348
max,0.135038,0.076125,0.157854,0.030987,0.101192,0.111774,0.147497,0.173821,0.099808,0.115573,...,0.119518,0.197589,0.113135,0.136490,0.162921,0.099021,0.086653,0.158176,0.166968,0.048967


In [36]:
%%timeit

min_for_arr = arr_r_less.min(axis = 0)
max_for_arr = arr_r_less.max(axis = 0)
columns_for_df = ['x{}'.format(i) for i in range(len(max_for_arr.compute()))]
pd.DataFrame(data = [min_for_arr.compute(), max_for_arr.compute()],
             columns = columns_for_df,
             index = ['min', 'max'])

1.6 s ± 75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
%%timeit

min_for_arr = arr_r_less.min(axis = 0)
max_for_arr = arr_r_less.max(axis = 0)
columns_for_df = ['x{}'.format(i) for i in range(len(max_for_arr.compute()))]
pd.DataFrame(data = dask.compute(min_for_arr, max_for_arr),
             columns = columns_for_df,
             index = ['min', 'max'])

1.66 s ± 310 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


4\. Найдите вектор $x \ne x_{256}$ из набора данных, ближайший к вектору $x_{256}$ в смысле метрики $L_1$. Выведите на экран первые 10 координат вектора $x$.

$$d_1(\textbf{x},\textbf{y})=\sum_{k=1}^{n}{|x_k - y_k|}, \textbf{x}, \textbf{y} \in \mathbb{R}^n$$

In [14]:
from scipy import spatial

In [9]:
x_256 = arr_r_less[:, [256]]
x_without_256 = arr_r_less[:, list(range(256))+list(range(257, arr_r_less.shape[1]))]

In [68]:
x_256

dask.array<getitem, shape=(1200000,), dtype=float32, chunksize=(100000,), chunktype=numpy.ndarray>

In [70]:
x_without_256.T

dask.array<transpose, shape=(311, 1200000), dtype=float32, chunksize=(311, 100000), chunktype=numpy.ndarray>

In [16]:
tree = spatial.KDTree(x_without_256.T)
tree.query(x_256)

(34.60950135438729, 181)

In [10]:
da.sum(da.abs(x_256 - x_without_256), axis=0).argmin().compute()

181

In [17]:
arr_r_less[:, 181].compute()[:10]

array([-0.03620593, -0.02302776, -0.02423452, -0.04748494, -0.03022445,
       -0.00947769,  0.01703103, -0.00542553,  0.01890532, -0.00433748],
      dtype=float32)

5\. Рецепты разбиты на 4 группы. Загрузите маску для разбиения на группы из датасета `mask` из файла `recipe_embeddings.h5` в виде `dask.array`. Для каждой группы посчитайте и выведите на экран максимальное значение  нормы $\ell_1$ векторов рецептов, принадлежащих к этой группе. 

Подсказка: закодируйте маску принадлежности к группе при помощи метода кодирования one-hot encoding и воспользуйтесь механизмом распространения.

$$\ell_1: ||\textbf{x}||_1=\sum_{k=1}^{n}{|x_k|}, \textbf{x} \in \mathbb{R}^n$$

In [18]:
dset_mask = hdf['mask']
arr_mask = da.from_array(dset_mask)

In [19]:
arr_mask.compute()

array([0, 0, 1, ..., 0, 1, 0])

In [20]:
from sklearn.preprocessing import OneHotEncoder

In [21]:
enc = OneHotEncoder(sparse=False)
enc = enc.fit_transform(arr_mask.reshape(-1, 1))

In [23]:
sums_by_groups = da.sum(enc.T.reshape((enc.shape[1], arr_mask.shape[0], 1)) * da.abs(arr).reshape(1, *arr.shape), axis=(1, 2))

In [25]:
sums_by_groups.compute()

array([7239407.21649696, 4826370.75549789, 2292290.11977783,
        120679.39365272])

6\. Пусть $X=[\textbf{x}_1,...\textbf{x}_M]^\top$ - матрица эмбеддингов рецептов размера $M\times N$, $W=[\textbf{w}_1,...,\textbf{w}_N]^\top$ - матрица коэффициентов некоторой модели машинного обучения размера $N\times 4$, $y=[y_1,...,y_M]^\top$ - вектор размера $M$, содержащий номера групп рецептов (метки классов). Тогда задачу классификации можно решить следующим образом: $$\hat{y_i} = argmax_j{<X_{i\cdot}, W_{\cdot j}>}$$ где $A_{i\cdot}$ обозначает $i$ строку матрицы, $A_{\cdot j}$ обозначает $j$ столбец матрицы, $\hat{y_i}$ - прогноз класса для рецепта $i$, $<\cdot, \cdot>$ - скалярное произведение векторов.

Инициализируйте матрицу $W$ случайным образом и получите прогнозы для всех рецептов при помощи этой матрицы и матрицы эмбеддингов. Подсчитайте и выведите на экран значение accuracy на основе полученных прогнозов $\hat{y}$ и правильных ответов $y$.

In [16]:
from dask_ml.metrics import accuracy_score

In [19]:
W = da.random.normal(1, 0, size = (arr_r_less.shape[1], 4))
accuracy_score(arr_mask, (arr_r_less @ W).argmax(axis = 1))

0.5

7\. Сингулярным разложением (SVD) матрицы $A$ размера $M\times N$ называется разложение вида $A = USV^\top$, где $U$ - матрица размера $M\times N$  ортонормированных векторов произведения $AA^\top$, $V^T$ - транспонированная матрица размера $N\times N$ ортонормированных векторов произведения $A^\top A$, $S$ - диагональная матрица сингулярных значений размера $N\times N$.

SVD может быть использовано для понижения размерности векторов. Для этого от матрицы $U$ оставляют первые $k$ столбцов $U_{\cdot,:k}$, от матрицы $S$ оставляют левый верхний квадрат размера $k\times k$ $S_{:k,:k}$ и вычисляется произведение $\hat{A} = U_{\cdot,:k}S_{:k,:k}$

Выберите эмбеддинги тех рецептов, которые относятся к группе с номеров 3, и уменьшите их размерность до 64 при помощи реализации алгоритма SVD из пакета `dask.array.linalg`. Выведите количество строк и столбцов полученного массива.

Примечание: после отбора рецепта, принадлежащих третьей группе, вызовите у полученного массива метод `compute_chunk_sizes`, чтобы `dask` обновил метаинформацию в этом массиве. 

In [21]:
third_group = arr_r_less[arr_mask == 3]
third_group.compute_chunk_sizes()

dask.array<getitem_variadic, shape=(10000, 312), dtype=float32, chunksize=(856, 312), chunktype=numpy.ndarray>

In [22]:
u, s, v = da.linalg.svd(third_group)

In [23]:
lower_dimension = u[:, :64] @ da.diag(s[:64])
lower_dimension

dask.array<getitem, shape=(10000, 64), dtype=float32, chunksize=(856, 64), chunktype=numpy.ndarray>

8\. Используя эмбеддинги уменьшенной размерности, полученные в задании 7, посчитайте косинусное сходство между каждой парой рецептов третьей группы. Выведите матрицу косинусного сходства на экран.

In [27]:
import dask_distance

In [28]:
matrix = 1 - dask_distance.cosine(lower_dimension, lower_dimension)
matrix.compute()

/Users/nastya/opt/anaconda3/lib/python3.9/site-packages/dask/array/core.py:4806: PerformanceWarning: Increasing number of chunks by factor of 12
  result = blockwise(
/Users/nastya/opt/anaconda3/lib/python3.9/site-packages/dask/array/core.py:4806: PerformanceWarning: Increasing number of chunks by factor of 12
  result = blockwise(


array([[1.        , 0.77996184, 0.70962796, ..., 0.78566269, 0.72498327,
        0.70748986],
       [0.77996184, 1.        , 0.64223863, ..., 0.86709919, 0.61588691,
        0.65344286],
       [0.70962796, 0.64223863, 1.        , ..., 0.64595621, 0.87130849,
        0.87150263],
       ...,
       [0.78566269, 0.86709919, 0.64595621, ..., 1.        , 0.64296757,
        0.66372941],
       [0.72498327, 0.61588691, 0.87130849, ..., 0.64296757, 1.        ,
        0.9126163 ],
       [0.70748986, 0.65344286, 0.87150263, ..., 0.66372941, 0.9126163 ,
        1.        ]])

9\. Используя результат задания 8, посчитайте и выведите на экран количество рецептов, для которых рецепт с индексом `242` входит число топ-5 ближайших рецептов в смысле косинусной близости. При поиске топ-5 рецептов для конкретного рецепта считайте, что он сам в это число не входит.

In [30]:
top = matrix[242].argtopk(6)
top[top != 242].compute()

array([4237, 2449, 7470,  768, 3286])

10\. Графом называется совокупность двух множеств $G=(V,E)$: множества $V=\{v_1, ..., v_M\}$ узлов и множества ребер $E=\{(v_i, v_j)|v_i\in V, v_j\in V\}$, соединяющих эти узлы. Матрицей смежности невзвешенного графа называется квадратная матрица $A=[a_{ij}]$, в которой ${a_{ij}}$ обозначает количество ребер, соединяющих вершины $i$ и $j$.

Постройте матрицу смежности для графа рецептов на основе матрицы косинусного сходства между каждой парой рецептов. Будем считать, что между двумя рецептами в этом графе существует ребро, если косинусное сходство между двумя этими рецептами не менее 0.85. Петли (ребра из вершины в саму в себя) в графе должны отсутствовать. Посчитайте и выведите на экран количество ребер в данном графе. Проверьте, является ли полученная матрица смежности симметричной.

Примечание: считайте, что два различных рецепта не могут иметь косинусное сходство, равное 1.

In [32]:
contiguity = da.where(matrix >= 0.85, 1, 0)
contiguity

dask.array<where, shape=(10000, 10000), dtype=int64, chunksize=(856, 856), chunktype=numpy.ndarray>

In [33]:
(contiguity == contiguity.T).all().compute()

True

In [34]:
contiguity.sum().compute()

23842782

11\. Работая с исходным файлом в формате `h5`, реализуйте алгоритм подсчета среднего вектора датасета в блочной форме.

Блочный алгоритм вычислений состоит из двух частей:
1. Загрузка фрагмента за фрагментом данных и проведение вычислений над этим фрагментом
2. Агрегация результатов вычислений на различных фрагментах для получения результата на уровне всего набора данных

Важно: при работе с `h5` в память загружаются не все элементы, а только те, которые запрашиваются в данный момент. При работе с `h5` вы можете работать с массивами `numpy.array`. Для итерации по сегментам файла допускается использование циклов.

Сравните время решения работы вашего алгоритма с реализацией поиска среднего вектора из `dask`. 

In [37]:
def mean_calculating(array, x_chunks=10, y_chunks=10):
    
    rows = np.linspace(0, array.shape[0]-1, y_chunks).astype(int)
    cols = np.linspace(0, array.shape[1]-1, x_chunks).astype(int)
    
    all_sums = []
    all_counts = []
    
    for row in range(1, len(rows)):
        for col in range(1, len(cols)):
            all_counts.append(array[rows[row-1]:rows[row], cols[col-1]:cols[col]].size)
            all_sums.append(array[rows[row-1]:rows[row], cols[col-1]:cols[col]].sum())
            
    return sum(all_sums) / sum(all_counts)

In [38]:
%%timeit
mean_calculating(dset)

5.2 s ± 93.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
%%timeit

arr_r_less.mean().compute()

502 ms ± 9.55 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
